In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('chemical_db.csv')

In [2]:
chem_list = ['Benzene', 'Cyclohexane', 'Toluene']
liq_volume =  8450.34 
chem_mass_fraction = [0.8869, 0.0318, 0.0822]

# every 3,171 lb of mixture) 2,812 lb of benzene, 258 lb of toluene, and 101 lb of cyclohexane
PhH = 2812/3171
PhMe = 258/3171
CyH = 101/3171
print('PhH: {} // PhMe: {} // CyH: {}'.format(PhH, PhMe, CyH))

PhH: 0.8867865026805424 // PhMe: 0.08136234626300852 // CyH: 0.03185115105644907


In [3]:
name_ = np.array([df[df['NAME'].isin(chem_list)]['NAME'].tolist()])
cas_ = np.array([df[df['NAME'].isin(chem_list)]['CAS'].tolist()])
mw_ = np.array([df[df['NAME'].isin(chem_list)]['MOLWT'].tolist()])
den_ = np.array([df[df['NAME'].isin(chem_list)]['L_DENS'].tolist()])
vp_a = np.array([df[df['NAME'].isin(chem_list)]['VP_COEF_A'].tolist()])
vp_b = np.array([df[df['NAME'].isin(chem_list)]['VP_COEF_B'].tolist()])
vp_c = np.array([df[df['NAME'].isin(chem_list)]['VP_COEF_C'].tolist()])
arr = np.concatenate((name_, cas_, mw_, den_, vp_a, vp_b, vp_c), axis=0).T

df3 = pd.DataFrame(data=arr, columns=['component',
                                      'cas_no',
                                      'mw',
                                      'den',
                                      'antoine_coef_a',
                                      'antoine_coef_b',
                                      'antoine_coef_c'])
df3

,component,cas_no,mw,den,antoine_coef_a,antoine_coef_b,antoine_coef_c
0,Benzene,00071-43-2,78.11,7.365,6.905,1211.033,220.79
1,Cyclohexane,00110-82-7,84.15999999999998,6.522,6.841,1201.53,222.65
2,Toluene,00108-88-3,92.13,7.261,6.954,1344.8,219.48


In [4]:
df3['tank_name'] = 'Tank 10001'
df3['tank_type'] = 'Vertical'
df3['facility_name'] = 'World Oil Refining'

df2 = pd.DataFrame({'component': chem_list, 'comp_mass_xi': chem_mass_fraction})

df1 = pd.merge(df2, df3, on='component', how='inner')

df1 = df1[['facility_name', 'tank_name', 'tank_type', 'component',
           'cas_no', 'mw', 'den', 'comp_mass_xi',
           'antoine_coef_a', 'antoine_coef_b', 'antoine_coef_c']]

In [5]:
df1['comp_liq_den'] = (df1['comp_mass_xi'].astype(float) / df1['den'].astype(float))*100
liq_density = np.sum(df1['comp_liq_den'])
liq_mass = liq_volume / liq_density
df1['comp_mass'] = df1['comp_mass_xi'].astype(float) * liq_mass
df1

,facility_name,tank_name,tank_type,component,cas_no,mw,den,comp_mass_xi,antoine_coef_a,antoine_coef_b,antoine_coef_c,comp_liq_den,comp_mass
0,World Oil Refining,Tank 10001,Vertical,Benzene,00071-43-2,78.11,7.365,0.8869,6.905,1211.033,220.79,12.042091,548.583324
1,World Oil Refining,Tank 10001,Vertical,Cyclohexane,00110-82-7,84.15999999999998,6.522,0.0318,6.841,1201.53,222.65,0.487580,19.669579
2,World Oil Refining,Tank 10001,Vertical,Toluene,00108-88-3,92.13,7.261,0.0822,6.954,1344.8,219.48,1.132075,50.844006


In [6]:
df1['liq_frac'] = df1['comp_mass'] / df1['den'].astype(float)

In [7]:
df1['liq_frac'].sum()

84.50340000000001

In [8]:
df1

,facility_name,tank_name,tank_type,component,cas_no,mw,den,comp_mass_xi,antoine_coef_a,antoine_coef_b,antoine_coef_c,comp_liq_den,comp_mass,liq_frac
0,World Oil Refining,Tank 10001,Vertical,Benzene,00071-43-2,78.11,7.365,0.8869,6.905,1211.033,220.79,12.042091,548.583324,74.485176
1,World Oil Refining,Tank 10001,Vertical,Cyclohexane,00110-82-7,84.15999999999998,6.522,0.0318,6.841,1201.53,222.65,0.487580,19.669579,3.015881
2,World Oil Refining,Tank 10001,Vertical,Toluene,00108-88-3,92.13,7.261,0.0822,6.954,1344.8,219.48,1.132075,50.844006,7.002342
